In [1]:
%load_ext autoreload
%autoreload 2

import sys 
import os

dep_paths = [
    '/fsx_0/user/tranx',
    '/fsx_0/user/tranx/llm_mm_aligner/replicated'
]

for p in dep_paths:
    if p not in sys.path:
        sys.path.append(p)

print("Using conda env:", os.environ['CONDA_DEFAULT_ENV'])

import torch 
print("Pytorch version:", torch.__version__)
print("Available GPUs:")
for i in range(torch.cuda.device_count()):
    print(f"Device {i}: {torch.cuda.get_device_name(i)}")


Using conda env: aligner_v7
Pytorch version: 2.3.0
Available GPUs:


/opt/hpcaas/.mounts/fs-036153e63d56f4dc2/home/tranx/conda/envs/aligner_v7/lib/python3.10/site-packages/torch/cuda/__init__.py:624: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


# Import

In [6]:
import ast
import json
import logging
import os
import sys
from dataclasses import replace
from itertools import chain
from typing import Any, Optional, Tuple

import torch

import transformers  # @manual=fbsource//third-party/pypi/transformers:transformers

from llm_mm_aligner.lib.callbacks.train import (
    tensorboard_callback,
    trainer_callback_factory_builder,
)

from llm_mm_aligner.lib.checkpoints import (
    checkpoint_downloader_factory,
    checkpoint_utils,
)
# from llm_mm_aligner.lib.clip_trainer import CLIPFSDPTrainer
from llm_mm_aligner.lib.configs import (
    DataTrainingArguments,
    ModelArguments,
    TrainingArguments,
)

from llm_mm_aligner.lib.data_collators import get_collator
from llm_mm_aligner.lib.datasets.hive import get_hive_dataset
from llm_mm_aligner.lib.datasets.mm_datasets import DatasetType, get_mm_dataset
from llm_mm_aligner.lib.datasets.web_dataset import get_wb_dataset
from llm_mm_aligner.lib.file_utils import (
    move_model_resources_to_local,
    set_max_manifold_parallel,
)
from llm_mm_aligner.lib.metaformers_bridge import model_parallel_cuda_manual_seed
from llm_mm_aligner.lib.models.clip_model import MMModelForCLIP
from llm_mm_aligner.lib.models.mm_model import MMModel
from llm_mm_aligner.lib.mpu import (
    get_model_parallel_group,
    get_pipeline_model_parallel_rank,
    initialize_model_parallel,
)
from llm_mm_aligner.lib.preemption import preemption_monitor_factory
from llm_mm_aligner.lib.profilers import profilers
from llm_mm_aligner.lib.trainers import (
    DPOTrainer,
    FSDPTrainer,
    MMLLMBaseTrainer,
    SimPOTrainer,
)
from llm_mm_aligner.lib.utils import (
    clamp,
    get_global_rank,
    get_local_process_group,
    get_local_rank,
    k_at_a_time,
)

from torch import distributed as dist
from torch.utils.tensorboard import SummaryWriter

# @manual=fbsource//third-party/pypi/transformers:transformers
from transformers import HfArgumentParser, set_seed, TrainerCallback

/opt/hpcaas/.mounts/fs-04cb39659581b642e/home/tranx/conda/envs/aligner_v1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
fused sequence parallel available


moe_matmul not found


# Load args for main()

In [38]:
import pickle

arg_path = "/data/home/yastashonok/TEMP/pickle/"

with open(os.path.join(arg_path, "model_args.pkl"), "rb") as f:
    model_args = pickle.load(f)

with open(os.path.join(arg_path, "data_args.pkl"), "rb") as f:
    data_args = pickle.load(f)

with open(os.path.join(arg_path, "training_args.pkl"), "rb") as f:
    training_args = pickle.load(f)

tb_writer = None

# Adapt path to AWS environment
base_path = "/data/home/yastashonok/mm6_pretrain"
data_path = "/data/home/yastashonok/MH13-aws/data"
model_path = "/data/home/yastashonok/MH13-aws/input"
run_name = "mm6_pretrain"

data_args.base_path = base_path
data_args.wd_data_path = data_path

model_args.model_name_or_path = f"{model_path}/Meta-Llama-3-8B-Instruct"
model_args.tokenizer_path = f"{model_path}/Meta-Llama-3-8B-Instruct"
model_args.modality_tokenizer_name = f"{model_path}/MetaCLIP-BigG-336-f536822271-8k"
training_args.output_dir = f"{base_path}/output/{run_name}"
training_args.run_name = run_name

In [40]:
parse_training_args = {'output_dir': '/data/home/yastashonok/mm6_pretrain/output/mm6_pretrain', 'overwrite_output_dir': True, 'do_train': True, 'do_eval': False, 'do_predict': False, 'eval_strategy': 'no', 'prediction_loss_only': False, 'per_device_train_batch_size': 1, 'per_device_eval_batch_size': 1, 'per_gpu_train_batch_size': None, 'per_gpu_eval_batch_size': None, 'gradient_accumulation_steps': 1, 'eval_accumulation_steps': None, 'eval_delay': 0, 'learning_rate': 0.0001, 'weight_decay': 0.1, 'adam_beta1': 0.9, 'adam_beta2': 0.95, 'adam_epsilon': 1e-08, 'max_grad_norm': 1.0, 'num_train_epochs': 1.0, 'max_steps': -1, 'lr_scheduler_type': 'cosine', 'lr_scheduler_kwargs': {}, 'warmup_ratio': 0.0, 'warmup_steps': 200, 'log_level': 'info', 'log_level_replica': 'info', 'log_on_each_node': True, 'logging_dir': '/data/home/yastashonok/mm6_pretrain/output/mm6_pretrain/runs/Jul19_15-08-44_h100-st-p548xlarge-36', 'logging_strategy': 'steps', 'logging_first_step': True, 'logging_steps': 10, 'logging_nan_inf_filter': True, 'save_strategy': 'steps', 'save_steps': 250, 'save_total_limit': None, 'save_safetensors': True, 'save_on_each_node': False, 'save_only_model': False, 'restore_callback_states_from_checkpoint': False, 'no_cuda': False, 'use_cpu': False, 'use_mps_device': False, 'seed': 2024, 'data_seed': None, 'jit_mode_eval': False, 'use_ipex': False, 'bf16': True, 'fp16': False, 'fp16_opt_level': 'O1', 'half_precision_backend': 'auto', 'bf16_full_eval': False, 'fp16_full_eval': False, 'tf32': None, 'local_rank': 7, 'ddp_backend': None, 'tpu_num_cores': None, 'tpu_metrics_debug': False, 'debug': [], 'dataloader_drop_last': False, 'eval_steps': 250.0, 'dataloader_num_workers': 0, 'dataloader_prefetch_factor': None, 'past_index': -1, 'run_name': 'mm6_pretrain', 'disable_tqdm': False, 'remove_unused_columns': False, 'label_names': None, 'load_best_model_at_end': False, 'metric_for_best_model': 'eval_loss', 'greater_is_better': False, 'ignore_data_skip': False, 'fsdp': ['full_shard', 'auto_wrap'], 'fsdp_min_num_params': 0, 'fsdp_config': {'forward_prefetch': False, 'backward_prefetch': 'backward_pre', 'limit_all_gathers': True, 'transformer_layer_cls_to_wrap': ['DecoderLayer',
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             'EncoderLayer', 'PerceptionTokenizer', 'MetaFormersBlock'], 'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False}, 'fsdp_transformer_layer_cls_to_wrap': None, 'accelerator_config': {'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None}, 'deepspeed': None, 'label_smoothing_factor': 0.0, 'optim': 'adamw_torch', 'optim_args': None, 'adafactor': False, 'group_by_length': False, 'length_column_name': 'length', 'report_to': ['tensorboard'], 'ddp_find_unused_parameters': None, 'ddp_bucket_cap_mb': None, 'ddp_broadcast_buffers': None, 'dataloader_pin_memory': False, 'dataloader_persistent_workers': False, 'skip_memory_metrics': True, 'use_legacy_prediction_loop': False, 'push_to_hub': False, 'resume_from_checkpoint': None, 'hub_model_id': None, 'hub_strategy': 'every_save', 'hub_token': '<HUB_TOKEN>', 'hub_private_repo': False, 'hub_always_push': False, 'gradient_checkpointing': False, 'gradient_checkpointing_kwargs': None, 'include_inputs_for_metrics': False, 'eval_do_concat_batches': True, 'fp16_backend': 'auto', 'evaluation_strategy': 'no', 'push_to_hub_model_id': None, 'push_to_hub_organization': None, 'push_to_hub_token': '<PUSH_TO_HUB_TOKEN>', 'mp_parameters': '', 'auto_find_batch_size': False, 'full_determinism': False, 'torchdynamo': None, 'ray_scope': 'last', 'ddp_timeout': 1800, 'torch_compile': False, 'torch_compile_backend': None, 'torch_compile_mode': None, 'dispatch_batches': None, 'split_batches': None, 'include_tokens_per_second': False, 'include_num_input_tokens_seen': False, 'neftune_noise_alpha': None, 'optim_target_modules': None, 'batch_eval_metrics': False, 'save_top_k': 1, 'load_latest_k': 1, 'checkpoints_optimizer': None, 'checkpoints_scheduler': None, 'dpo_alpha': 0.0, 'dpo_beta': 0.1, 'dpo_reference_free': False, 'dpo_label_smoothing': 0.0, 'ipo': False, 'simpo_beta': 2.0, 'simpo_gamma': 1.0, 'mixin_gamma': 1.0, 'load_best_model': False, 'clip_head_weight_decay': None, 'lm_text_lr_scale': 1.0, 'perception_lr_scale': 1.0, 'tb_logdir': None, 'eval_ckpt': None, 'eval_type': None, 'cosine_decay_to': 0.01}
parse_model_args = {'model_name_or_path': '/data/home/yastashonok/MH13-aws/input/Meta-Llama-3-8B-Instruct', 'tokenizer_path': '/data/home/yastashonok/MH13-aws/input/Meta-Llama-3-8B-Instruct', 'use_metaformers': True, 'recompute_attn': True, 'recompute_fc1_fc3': True, 'pretraining_tp': 1, 'model_parallel_size': 1, 'context_parallel_size': 1, 'use_te': False, 'parallelize_llm': True, 'parallelize_perception_tokenizer': True, 'use_sdpa': True, 'use_fused_layernorm': True, 'max_parallel_model_loading': 8, 'model_type': None, 'config_overrides': None, 'config_name': None, 'lm_mh_tokenizer_version': 'tiktoken_v5', 'tokenizer_name': None, 'ensemble_tokenizer_params': '{}', 'use_ensemble_modality_tokenizer': False, 'modality_tokenizer_name': '/data/home/yastashonok/MH13-aws/input/MetaCLIP-BigG-336-f536822271-8k', 'cache_dir': None, 'use_fast_tokenizer': True, 'model_revision': 'main', 'n_prefix_embs': 65, 'seq2seq': False, 'num_classes': None, 'routing_classifier': False, 'integrity_classifier': False, 'freeze_perception': True, 'freeze_perception_except_pos_emb': False, 'ignore_mismatched_sizes': False, 'freeze_tokenizer': False, 'freeze_lm': True, 'lm_bits': -1, 'checkpoints_lm': None, 'perception_tokenizer_name': None, 'checkpoints_perception': None, 'checkpoints_perception_tokenizer': None, 'checkpoints_logit_scale': None, 'checkpoints_logit_bias': None, 'tokenizer_type': 'PerceiverV3', 'modality': 'image', 'mixin_modality': 'image', 'lora': False, 'lora_rank': 8, 'lora_dropout': 0.1, 'lora_alpha': 32, 'lora_checkpoint': None, 'lora_target_modules':
                    "['q_proj', 'v_proj']", 'lora_tokenizer': False, 'lora_tokenizer_rank': 8, 'lora_tokenizer_dropout': 0.1, 'lora_tokenizer_alpha': 32, 'lora_tokenizer_checkpoint': None, 'lora_tokenizer_target_modules': "['to_q', 'to_kv']", 'lora_perception': False, 'lora_perception_rank': 8, 'lora_perception_dropout': 0.1, 'lora_perception_alpha': 32, 'lora_perception_checkpoint': None, 'lora_perception_target_modules': "['k_proj', 'v_proj', 'q_proj', 'out_proj', 'fc1', 'fc2', 'visual_projection']", 'speech_adapter_type': 0, 'num_media_embeds': 4, 'perception_tokenizer_num_layers': 8, 'perception_tokenizer_project_in_dim': None, 'perception_tokenizer_project_out_from_dim': None, 'perception_tokenizer_attention_dropout_p': 0.05, 'perception_tokenizer_hidden_dropout_p': 0.1, 'perception_tokenizer_ensemble_type': 'pool', 'custom_FSDP': False, 'device_map': 'cpu', 'load_in_8bit': False, 'low_cpu_mem_usage': True, 'add_bos_token': False, 'add_eos_token': False, 'manifold_bucket': None, 'perception_output_processor': 'layer', 'keep_vision_hidden_states': False, 'vision_hidden_state_layer': -2, 'gradient_checkpointing_llm': True, 'layer_ckpt': 'none', 'gradient_checkpointing_perception_tokenizer': True, 'gradient_checkpointing_perception': False, 'clip_patch_dropout': None, 'center_crop': False, 'clip_loss_weight': None, 'clip_logit_scale': 2.6592, 'clip_n_registers': None, 'clip_enable_mi': False, 'debug_logs': False, 'llama_seq_cls_dropout_p': None, 'add_grounding_tokens': False, 'grounding_bin_size': 42, 'grounding_image_size': 336, 'add_grounding_llm_head_if_available': True, 'checkpoints_lm_embedding': None, 'checkpoints_lm_head': None, 'checkpoints_lm_cls_head': None, 'perceiver_num_heads': 32, 'perceiver_dim_override': 4096, 'perceiver_num_kv_heads': 8, 'perceiver_num_latents': 64, 'perceiver_source_num_latents': 64, 'perceiver_ff_mult': 4, 'perceiver_use_pos_embs': True, 'perceiver_pos_emb_type': 'learned', 'perceiver_enable_query_aware': False, 'clip_num_embeddings': 577, 'encoder_frame_stride': None, 'mi_image_encoder_frame_stride': None, 'encoder_num_frames': None, 'encoder_use_predictor': True, 'perceiver_cat_latents': True, 'perceiver_collapse_chunks': False, 'perceiver_enable_mi': False, 'treat_frames_as_images': False, 'perceiver_max_img_frames': 16, 'block_sparse': True, 'perceiver_num_frames_per_chunk': 4, 'perceiver_add_eof': True, 'perceiver_collapse_frame_chunks': False, 'perceiver_dynamic_frame_resizing': True, 'perceiver_enable_latent_compression': False, 'perceiver_interleaved_outputs': False, 'perceiver_collapse_modalities': False, 'replace_llama_attention': True, 'ensemble_meta_perceiver_option': None, 'ensemble_freeze_modality': None, 'enable_multi_transformer_blocks': False, 'n_multi_transformer_blocks': 35, 'n_layers_per_multi_transformer_block': 2, 'enable_modality_aggregator': False, 'modality_aggregator_split_dim': 2, 'modality_aggregator_merge_dim': 1, 'perceiver_enable_moe': False, 'perceiver_num_experts': 8, 'perceiver_num_activated_experts': 2, 'use_scaled_rope': True, 'rope_scale_factor': 8, 'high_freq_factor': 32, 'low_freq_factor': 1}

In [63]:
training_args.fsdp_config['fsdp_transformer_layer_cls_to_wrap'] = [
    'DecoderLayer', 'EncoderLayer', 'PerceptionTokenizer', 'MetaFormersBlock']

In [70]:
kw = {
    'process_group': '<torch.distributed.distributed_c10d.ProcessGroup object at 0x7f98d0b26930>', 
    'sharding_strategy': "<ShardingStrategy.FULL_SHARD: 1>"," 
    'auto_wrap_policy': "functools.partial(<function transformer_auto_wrap_policy at 0x7f9b74cff7f0>", 
    transformer_layer_cls={<class 'llm_mm_aligner.lib.tokenizers.mm_tokenizer.PerceptionTokenizer'>, 
        MetaLayer(infix='EncoderLayer'), 
        <class 'llm_mm_aligner.metaformers_clone.src.llama3.model.transformer.TransformerBlock'>, 
        MetaLayer(infix='DecoderLayer')}), 
        'cpu_offload': CPUOffload(offload_params=False), 
        'mixed_precision': MixedPrecision(param_dtype=torch.bfloat16, reduce_dtype=torch.bfloat16, buffer_dtype=None, keep_low_precision_grads=False, cast_forward_inputs=True, cast_root_forward_inputs=True, _module_classes_to_ignore=(<class 'torch.nn.modules.batchnorm._BatchNorm'>,)), 
        'device_id': device(type='cuda', index=3), 
        'backward_prefetch': 'backward_pre', 'forward_prefetch': False, 'limit_all_gathers': True, 'use_orig_params': True}

SyntaxError: unterminated string literal (detected at line 1) (1916843494.py, line 1)

In [64]:
parse_model_args['custom_FSDP']

False

In [60]:
# parse_training_args['fsdp_config']['fsdp_transformer_layer_cls_to_wrap']
parse_training_args['fsdp_config']['transformer_layer_cls_to_wrap']

['DecoderLayer', 'EncoderLayer', 'PerceptionTokenizer', 'MetaFormersBlock']

In [50]:
training_args.accelerator_config.use_configured_state = False

In [19]:
training_args.accelerator_config.to_dict()

{'split_batches': False,
 'dispatch_batches': None,
 'even_batches': True,
 'use_seedable_sampler': True,
 'non_blocking': False,
 'gradient_accumulation_kwargs': None}

In [26]:
s = {'output_dir': '/data/home/yastashonok/mm6_pretrain/output/mm6_pretrain', 'overwrite_output_dir': True, 'do_train': True, 'do_eval': False, 'do_predict': False, 'eval_strategy': 'no', 'prediction_loss_only': False, 'per_device_train_batch_size': 1, 'per_device_eval_batch_size': 1, 'per_gpu_train_batch_size': None, 'per_gpu_eval_batch_size': None, 'gradient_accumulation_steps': 1, 'eval_accumulation_steps': None, 'eval_delay': 0, 'learning_rate': 0.0001, 'weight_decay': 0.1, 'adam_beta1': 0.9, 'adam_beta2': 0.95, 'adam_epsilon': 1e-08, 'max_grad_norm': 1.0, 'num_train_epochs': 1.0, 'max_steps': -1, 'lr_scheduler_type': 'cosine', 'lr_scheduler_kwargs': {}, 'warmup_ratio': 0.0, 'warmup_steps': 200, 'log_level': 'info', 'log_level_replica': 'info', 'log_on_each_node': True, 'logging_dir': '/data/home/yastashonok/mm6_pretrain/output/mm6_pretrain/runs/Jul19_14-55-59_h100-st-p548xlarge-36', 'logging_strategy': 'steps', 'logging_first_step': True, 'logging_steps': 10, 'logging_nan_inf_filter': True, 'save_strategy': 'steps', 'save_steps': 250, 'save_total_limit': None, 'save_safetensors': True, 'save_on_each_node': False, 'save_only_model': False, 'restore_callback_states_from_checkpoint': False, 'no_cuda': False, 'use_cpu': False, 'use_mps_device': False, 'seed': 2024, 'data_seed': None, 'jit_mode_eval': False, 'use_ipex': False, 'bf16': True, 'fp16': False, 'fp16_opt_level': 'O1', 'half_precision_backend': 'auto', 'bf16_full_eval': False, 'fp16_full_eval': False, 'tf32': None, 'local_rank': 4, 'ddp_backend': None, 'tpu_num_cores': None, 'tpu_metrics_debug': False, 'debug': [], 'dataloader_drop_last': False, 'eval_steps': 250.0, 'dataloader_num_workers': 0, 'dataloader_prefetch_factor': None, 'past_index': -1, 'run_name': 'mm6_pretrain', 'disable_tqdm': False, 'remove_unused_columns': False, 'label_names': None, 'load_best_model_at_end': False, 'metric_for_best_model': 'eval_loss', 'greater_is_better': False, 'ignore_data_skip': False, 'fsdp': ['full_shard', 'auto_wrap'], 'fsdp_min_num_params': 0, 'fsdp_config': {'forward_prefetch': False, 'backward_prefetch': 'backward_pre', 'limit_all_gathers': True, 'transformer_layer_cls_to_wrap': ['DecoderLayer',
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           'EncoderLayer', 'PerceptionTokenizer', 'MetaFormersBlock'], 'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False}, 'fsdp_transformer_layer_cls_to_wrap': None, 'accelerator_config': {'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None}, 'deepspeed': None, 'label_smoothing_factor': 0.0, 'optim': 'adamw_torch', 'optim_args': None, 'adafactor': False, 'group_by_length': False, 'length_column_name': 'length', 'report_to': ['tensorboard'], 'ddp_find_unused_parameters': None, 'ddp_bucket_cap_mb': None, 'ddp_broadcast_buffers': None, 'dataloader_pin_memory': False, 'dataloader_persistent_workers': False, 'skip_memory_metrics': True, 'use_legacy_prediction_loop': False, 'push_to_hub': False, 'resume_from_checkpoint': None, 'hub_model_id': None, 'hub_strategy': 'every_save', 'hub_token': '<HUB_TOKEN>', 'hub_private_repo': False, 'hub_always_push': False, 'gradient_checkpointing': False, 'gradient_checkpointing_kwargs': None, 'include_inputs_for_metrics': False, 'eval_do_concat_batches': True, 'fp16_backend': 'auto', 'evaluation_strategy': 'no', 'push_to_hub_model_id': None, 'push_to_hub_organization': None, 'push_to_hub_token': '<PUSH_TO_HUB_TOKEN>', 'mp_parameters': '', 'auto_find_batch_size': False, 'full_determinism': False, 'torchdynamo': None, 'ray_scope': 'last', 'ddp_timeout': 1800, 'torch_compile': False, 'torch_compile_backend': None, 'torch_compile_mode': None, 'dispatch_batches': None, 'split_batches': None, 'include_tokens_per_second': False, 'include_num_input_tokens_seen': False, 'neftune_noise_alpha': None, 'optim_target_modules': None, 'batch_eval_metrics': False, 'save_top_k': 1, 'load_latest_k': 1, 'checkpoints_optimizer': None, 'checkpoints_scheduler': None, 'dpo_alpha': 0.0, 'dpo_beta': 0.1, 'dpo_reference_free': False, 'dpo_label_smoothing': 0.0, 'ipo': False, 'simpo_beta': 2.0, 'simpo_gamma': 1.0, 'mixin_gamma': 1.0, 'load_best_model': False, 'clip_head_weight_decay': None, 'lm_text_lr_scale': 1.0, 'perception_lr_scale': 1.0, 'tb_logdir': None, 'eval_ckpt': None, 'eval_type': None, 'cosine_decay_to': 0.01}

In [32]:
training_args.eval_on_start

False

In [34]:
training_args.fsdp_config

{'forward_prefetch': False,
 'backward_prefetch': 'backward_pre',
 'limit_all_gathers': True,
 'transformer_layer_cls_to_wrap': ['DecoderLayer',
  'EncoderLayer',
  'PerceptionTokenizer',
  'MetaFormersBlock'],
 'min_num_params': 0,
 'xla': False,
 'xla_fsdp_v2': False,
 'xla_fsdp_grad_ckpt': False}

In [33]:
training_args.fsdp_config = {'forward_prefetch': False,
                             'backward_prefetch': 'backward_pre',
                             'limit_all_gathers': True,
                             'transformer_layer_cls_to_wrap': ['DecoderLayer',
                                                               'EncoderLayer',
                                                               'PerceptionTokenizer',
                                                               'MetaFormersBlock'],
                             'min_num_params': 0,
                             'xla': False,
                             'xla_fsdp_v2': False,
                             'xla_fsdp_grad_ckpt': False}

In [30]:
s["fsdp_config"]

{'forward_prefetch': False,
 'backward_prefetch': 'backward_pre',
 'limit_all_gathers': True,
 'transformer_layer_cls_to_wrap': ['DecoderLayer',
  'EncoderLayer',
  'PerceptionTokenizer',
  'MetaFormersBlock'],
 'min_num_params': 0,
 'xla': False,
 'xla_fsdp_v2': False,
 'xla_fsdp_grad_ckpt': False}

In [31]:
s["eval_on_start"]

KeyError: 'eval_on_start'

In [21]:
training_args.eval_on_start = False

In [10]:
meta_args = {
    "training_args": training_args.to_dict(),
    "model_args": model_args.to_dict(),
    "data_args": data_args.to_dict()
}

AttributeError: 'DataTrainingArguments' object has no attribute 'to_dict'

In [15]:
for item in data_args:
    print(item)

TypeError: 'DataTrainingArguments' object is not iterable

# main

## helpers

In [70]:
import llm_common

In [72]:
llm_common

<module 'llm_common' from '/fsx_0/user/tranx/xlformers/src/finetune/packages/llm_common/llm_common/__init__.py'>

In [40]:
def set_training_axioms(
    model_args: ModelArguments,
    data_args: DataTrainingArguments,
    training_args: TrainingArguments,
):
    """This function sets what should always be true for the training process"""
    if model_args.lora_tokenizer:
        model_args.parallelize_perception_tokenizer = False

In [58]:
set_training_axioms(model_args, data_args, training_args)

# Get GPU ranks
local_rank = get_local_rank()
global_rank = get_global_rank()

preemption_monitor_factory.create_preemption_monitor(global_rank)

mixin_model_args = replace(
    model_args,
    modality=model_args.mixin_modality,
)

mixin_data_args = replace(
    data_args,
    train_file=data_args.mixin_file,
    filetype=data_args.mixin_filetype,
    max_train_samples=None,
    use_hive_dataset=False,
    task_type=data_args.mixin_task_type,
    instr_prompt=data_args.mixin_instr_prompt,
    dataset_name="MMdataset",
)

# handle ensemble case
if model_args.use_ensemble_modality_tokenizer:
    # parse modality_tokenizer_name into a list of modality_tokenizers
    model_args.modality_tokenizer_name = ast.literal_eval(
        model_args.modality_tokenizer_name
    )

In [69]:
training_args.

AttributeError: 'TrainingArguments' object has no attribute 'auto_wrap_policy'

In [67]:
training_args.fsdp_config

{'min_num_params': 0,
 'xla': False,
 'xla_fsdp_v2': False,
 'xla_fsdp_grad_ckpt': False,
 'fsdp_transformer_layer_cls_to_wrap': ['DecoderLayer',
  'EncoderLayer',
  'PerceptionTokenizer',
  'MetaFormersBlock'],
 'forward_prefetch': False,
 'backward_prefetch': 'backward_pre',
 'limit_all_gathers': True}

In [65]:
import torch.distributed as dist
os.environ['RANK'] = '0'
os.environ['WORLD_SIZE'] = '8'
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '29500'

dist.init_process_group(backend='nccl')
torch.distributed.is_initialized()

DistStoreError: Timed out after 601 seconds waiting for clients. 1/8 clients joined.

In [56]:


initialize_model_parallel(
    tensor_model_parallel_size=model_args.model_parallel_size,
    pipeline_model_parallel_size=1,
    virtual_pipeline_model_parallel_size=None,
    pipeline_model_parallel_split_rank=None,
    use_fp8=False,
    alternate_pp_config=False,
    context_parallel_size=model_args.context_parallel_size,
    background_nccl_init=False,
    shared_kv_head_group_size=None,
)

AssertionError: 

# Pytorch - numpy

In [71]:
# error: Could not infer dtype of numpy.uint8

In [72]:
import torch
import numpy as np

In [75]:
np.__version__

'2.0.0'

In [73]:
t = torch.as_tensor([3, 4], dtype=torch.uint8)

In [74]:
t = torch.as_tensor([3, 4], dtype=np.uint8)

TypeError: as_tensor(): argument 'dtype' must be torch.dtype, not type